In [1]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf
import numpy as np
from ctc_segmentation import ctc_segmentation, prepare_text, CtcSegmentationParameters
import librosa
import pandas as pd
import os

/blue/arthur.porto-biocosmos/ahmed.waseem/.conda/envs/ctc_seg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("/blue/ufdatastudios/ahmed.waseem/ctc/meta_speaker.csv")

In [3]:
df = df[df['duration'].apply(lambda x: x >= 5)]
audio_folder = "/blue/ufdatastudios/ahmed.waseem/processed_audio"
df["audio_filepath"] = df["audio_filepath"].apply(lambda x: os.path.join(audio_folder, x))



In [4]:
df.columns

Index(['audio_filepath', 'duration', 'text', 'gender', 'age-group',
       'primary_language', 'native_place_state', 'native_place_district',
       'highest_qualification', 'job_category', 'occupation_domain'],
      dtype='object')

In [21]:
df.head()

,audio_filepath,duration,text,gender,age-group,primary_language,native_place_state,native_place_district,highest_qualification,job_category,occupation_domain
3852,/blue/ufdatastudios/ahmed.waseem/processed_aud...,5.0,and they went off into the space when earth wa...,Female,45-60,Telugu,Odisha,Sambalpur,Graduate,Full Time,Education and Research
3853,/blue/ufdatastudios/ahmed.waseem/processed_aud...,5.0,I should say addictive because once I started ...,Female,45-60,Telugu,Odisha,Sambalpur,Graduate,Full Time,Education and Research
3854,/blue/ufdatastudios/ahmed.waseem/processed_aud...,5.0,and with the kind of dishes that you have in I...,Female,18-30,Marathi,Maharashtra,Ratnagiri,Graduate,Part Time,Social service
3855,/blue/ufdatastudios/ahmed.waseem/processed_aud...,5.0,"So in order to bring all these back, in order ...",Female,18-30,Malayalam,Kerala,Pathanamthitta,Post Graduate,Full Time,Education and Research
3856,/blue/ufdatastudios/ahmed.waseem/processed_aud...,5.0,It's a well-known fact that India is a country...,Female,30-45,Hindi,Goa,South Goa,Post Graduate,Full Time,Technology and Services


In [4]:
df.columns

Index(['audio_filepath', 'duration', 'text', 'gender', 'age-group',
       'primary_language', 'native_place_state', 'native_place_district',
       'highest_qualification', 'job_category', 'occupation_domain'],
      dtype='object')

In [5]:
audio_filepaths = df['audio_filepath'].tolist()
transcripts = df['text'].tolist()
text = df['text'].tolist()

In [6]:
model_name = "facebook/wav2vec2-large-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=1024, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder

In [7]:
config = CtcSegmentationParameters()
config.blank = processor.tokenizer.pad_token_id
config.skip_prob = 1.0  
config.max_prob = 0.95  

In [8]:
def get_logits(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

    if waveform.ndim == 1:
        waveform = waveform.unsqueeze(0)  
    
    inputs = processor(waveform.tolist(), sampling_rate=16000, return_tensors="pt", padding=True)
    
    with torch.no_grad():
        logits = model(inputs.input_values.to(model.device)).logits.cpu()
    
    if logits.ndim == 3:
        logits = logits.squeeze(0)  
    
    return logits

In [14]:
aligned_data = []
for _, row in df.iterrows():
    audio_path = row["audio_filepath"]
    text = row["text"]
    
    logits = get_logits(audio_path)

    lpz = torch.nn.functional.log_softmax(logits, dim=-1).numpy()
    lpz = np.array(lpz, dtype=np.float32) 
    print("lpz shape:", lpz.shape) 

    ground_truth = processor.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids[0].tolist()
    ground_truth = np.array(ground_truth, dtype=np.int64)  
    print("Ground truth shape:", ground_truth.shape, "Content:", ground_truth)  

    try:
        timing, char_list = ctc_segmentation(config, lpz, ground_truth)
    except ValueError as e:
        print("Error during CTC segmentation:", e)
        continue 

    word_alignments = []
    current_word = ""
    word_start_time = None
    
    for idx, (start, end) in enumerate(zip(timing["begin"], timing["end"])):
        subword = processor.tokenizer.decode([ground_truth[idx]])
        
        if subword.strip() != "":
            if current_word == "":
                word_start_time = start
            
            current_word += subword
            
            if " " in text[len(" ".join([w["word"] for w in word_alignments])) :].strip().split(" ", 1)[0]:
                word_alignments.append({
                    "word": current_word.strip(),
                    "start_time": word_start_time,
                    "end_time": end
                })
                current_word = ""
                word_start_time = None
    
    aligned_data.append({"audio_filepath": audio_path, "text": text, "word_alignments": word_alignments})

aligned_df = pd.DataFrame(aligned_data)

aligned_df

lpz shape: (249, 32)
Ground truth shape: (77,) Content: [3 3 3 4 3 3 3 3 4 3 3 3 3 4 3 3 3 4 3 3 3 3 4 3 3 3 4 3 3 3 3 3 4 3 3 3 3
 4 3 3 3 3 3 4 3 3 3 4 3 3 3 3 3 3 3 4 3 3 4 3 3 4 3 3 3 3 3 4 3 3 3 3 3 3
 3 3 3]
Error during CTC segmentation: Buffer has wrong number of dimensions (expected 2, got 1)
lpz shape: (249, 32)
Ground truth shape: (92,) Content: [10  4  3  3  3  3  3  3  4  3  3  3  4  3  3  3  3  3  3  3  3  3  4  3
  3  3  3  3  3  3  4  3  3  3  3  4 10  4  3  3  3  3  3  3  3  4  3  3
  3  3  3  3  3  3  3  4 10  4  3  3  3  3  3  3 27  3  4  3  3  3  3  4
  3  3  4  3  3  3  3  3  4 10  4  3  3  3  3  3  3  3  3  3]
Error during CTC segmentation: Buffer has wrong number of dimensions (expected 2, got 1)
lpz shape: (249, 32)
Ground truth shape: (60,) Content: [ 3  3  3  4  3  3  3  3  4  3  3  3  4  3  3  3  3  4  3  3  4  3  3  3
  3  3  3  4  3  3  3  3  4  3  3  3  4  3  3  3  3  4  3  3  4 10  3  3
  3  3  3  4  3  3  3  3  3  3  3  3]
Error during CTC segmentation: 

KeyboardInterrupt: 

["'", '<', '>', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [33]:
print("Processor vocabulary size:", len(processor.tokenizer))

Processor vocabulary size: 32
